### Music Recommendation System Using Python

To get started with building a Music Recommendation System, we first need to have an access token. The access token serves as a temporary authorization credential, allowing the code to make authenticated requests to the Spotify API on behalf of the application.

The **CLIENT_ID** and **CLIENT_SECRET** variables hold my credentials that uniquely identify the application making requests to the Spotify API. The Client ID identifies the application, while the Client Secret is a confidential key used for authentication.

In [133]:
import requests
import base64

#My credentials that allow me to make requests using Spotify API
CLIENT_ID = '648ece1061bb46c0bc2c7e360700847c'
CLIENT_SECRET = '3a6447a30cee4d869cc32393ce0cfabb'

Next, the client ID and secret are combined in the client_credentials variable, separated by a colon (:). Then, this string is encoded using Base64 encoding to create a secure representation of the credentials. We then proceed to request an access token from the Spotify API.

In [134]:
client_credentials = f"{CLIENT_ID}:{CLIENT_SECRET}"
    
client_credentials_base64 = base64.b64encode(client_credentials.encode())

### Requesting token access from Spotify API

The Spotify API sends a POST request to the token url with the client credentials in the authorization header, this is required for client authentication.

The grant_type parameter is set to **client_credentials** to indicate that the application is requesting an access token for the client credentials flow.

With the access token, the application can now make authorized requests to retrieve music data, such as tracks, albums, artists, and user information, which is fundamental for building a music recommendation system using the Spotify API and Python.

In [137]:
#requesting access token
token_url = 'https://accounts.spotify.com/api/token'
headers = {
    'Authorization':f'Basic {client_credentials_base64.decode()}'
}

data = {
    'grant_type':'client_credentials'
}

response = requests.post(token_url, data = data, headers = headers)

if response.status_code == 200:
    access_token = response.json()['access_token']
    print("Access token obtained succcessfully.")
else:
    print("Error obtaining accesstoken.")
    exit()

Access token obtained succcessfully.


### Using Spotipy collect music data from any  playlist on Spotify

Spotipy is a lightweight Python library for the Spotify Web API. With Spotipy you get full access to all of the music data provided by the Spotify platform.

In [151]:
#importing important packages
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth

In [139]:
def get_trending_playlist_data(playlist_id, access_token):
    # Set up Spotipy with the access token
    sp = spotipy.Spotify(auth=access_token)

    # Get the tracks from the playlist
    playlist_tracks = sp.playlist_tracks(playlist_id, fields='items(track(id, name, artists, album(id, name)))')

    # Extract relevant information and store in a list of dictionaries
    music_data = []
    for track_info in playlist_tracks['items']:
        track = track_info['track']
        track_name = track['name']
        artists = ', '.join([artist['name'] for artist in track['artists']])
        album_name = track['album']['name']
        album_id = track['album']['id']
        track_id = track['id']

        # Get audio features for the track
        audio_features = sp.audio_features(track_id)[0] if track_id != 'Not available' else None

        # Get release date of the album
        try:
            album_info = sp.album(album_id) if album_id != 'Not available' else None
            release_date = album_info['release_date'] if album_info else None
        except:
            release_date = None

        # Get popularity of the track
        try:
            track_info = sp.track(track_id) if track_id != 'Not available' else None
            popularity = track_info['popularity'] if track_info else None
        except:
            popularity = None

        # Add additional track information to the track data
        track_data = {
            'Track Name': track_name,
            'Artists': artists,
            'Album Name': album_name,
            'Album ID': album_id,
            'Track ID': track_id,
            'Popularity': popularity,
            'Release Date': release_date,
            'Duration (ms)': audio_features['duration_ms'] if audio_features else None,
            'Explicit': track_info.get('explicit', None),
            'External URLs': track_info.get('external_urls', {}).get('spotify', None),
            'Danceability': audio_features['danceability'] if audio_features else None,
            'Energy': audio_features['energy'] if audio_features else None,
            'Key': audio_features['key'] if audio_features else None,
            'Loudness': audio_features['loudness'] if audio_features else None,
            'Mode': audio_features['mode'] if audio_features else None,
            'Speechiness': audio_features['speechiness'] if audio_features else None,
            'Acousticness': audio_features['acousticness'] if audio_features else None,
            'Instrumentalness': audio_features['instrumentalness'] if audio_features else None,
            'Liveness': audio_features['liveness'] if audio_features else None,
            'Valence': audio_features['valence'] if audio_features else None,
            'Tempo': audio_features['tempo'] if audio_features else None
        }

        music_data.append(track_data)

    # Create a pandas DataFrame from the list of dictionaries
    df = pd.DataFrame(music_data)

    return df

https://open.spotify.com/playlist/07NDTthdZ36aiaL6uGUQkK

In [140]:
playlist_id = '07NDTthdZ36aiaL6uGUQkK'

# Call the function to get the music data from the playlist and store it in a DataFrame
music_df = get_trending_playlist_data(playlist_id, access_token)

# Display the DataFrame
print(music_df)

                                           Track Name  \
0                                      Take My Breath   
1                                   Let There Be Rock   
2              Way To Break My Heart (feat. Skrillex)   
3         Hearts On Fire - From "Rocky IV" Soundtrack   
4   Tick Tick Boom (feat. BygTwo3) - From 'SCOOB!'...   
..                                                ...   
95                                            Forever   
96                                          Adore You   
97                                   Watermelon Sugar   
98                                    Blinding Lights   
99                                             No Lie   

                     Artists                      Album Name  \
0                 The Weeknd                         Dawn FM   
1                      AC/DC               Let There Be Rock   
2       Ed Sheeran, Skrillex     No.6 Collaborations Project   
3              John Cafferty                        Rocky I

In [141]:
#Exploratory Data Analysis
music_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Track Name        100 non-null    object 
 1   Artists           100 non-null    object 
 2   Album Name        100 non-null    object 
 3   Album ID          100 non-null    object 
 4   Track ID          100 non-null    object 
 5   Popularity        100 non-null    int64  
 6   Release Date      100 non-null    object 
 7   Duration (ms)     100 non-null    int64  
 8   Explicit          100 non-null    bool   
 9   External URLs     100 non-null    object 
 10  Danceability      100 non-null    float64
 11  Energy            100 non-null    float64
 12  Key               100 non-null    int64  
 13  Loudness          100 non-null    float64
 14  Mode              100 non-null    int64  
 15  Speechiness       100 non-null    float64
 16  Acousticness      100 non-null    float64
 17

In [142]:
# Building Music Reccomendation using Python
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity

#renaming music dataframe to data
data = music_df

In [143]:
#function to calculate the weighted popularity scores based on release date
def caluclated_weight_popularity(release_date):
    
    #converting the release date to datetime object
    release_date = datetime.strptime(release_date, '%Y-%m-%d')
    
    #Calculating the timespan between release date and today's date
    time_span = datetime.now() - release_date
    
    #calculate the weighted popularity score based on time span (e.g., more recent releases have higher weight)
    weight = 1/(time_span.days + 1)
    return weight

In [144]:
#Normaizing the music features using Min-Max scaling
scaler = MinMaxScaler()

music_features = data[['Danceability', 'Energy', 'Key', 
                           'Loudness', 'Mode', 'Speechiness', 'Acousticness',
                           'Instrumentalness', 'Liveness', 'Valence', 'Tempo']].values

music_features_scaled = scaler.fit_transform(music_features)

In [145]:
#creating recomendations based on music audio features
def content_based_recommendations(input_song_name, num_recommendations=5):
    if input_song_name not in data['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    # Get the index of the input song in the music DataFrame
    input_song_index = data[data['Track Name'] == input_song_name].index[0]

    # Calculate the similarity scores based on music features (cosine similarity)
    similarity_scores = cosine_similarity([music_features_scaled[input_song_index]], music_features_scaled)

    # Get the indices of the most similar songs
    similar_song_indices = similarity_scores.argsort()[0][::-1][1:num_recommendations + 1]

    # Get the names of the most similar songs based on content-based filtering
    content_based_recommendations = data.iloc[similar_song_indices][['Track Name', 'Artists', 'Album Name', 'Release Date', 'Popularity']]

    return content_based_recommendations

In [146]:
#Getting Reccomendations for 'Paint The Town Red' by Doja Cat.

content_based_recommendations('Paint The Town Red', 25)

,Track Name,Artists,Album Name,Release Date,Popularity
81,Butter,BTS,"Butter (Hotter, Sweeter, Cooler)",2021-06-04,72
37,Sympathy For The Devil - 50th Anniversary Edition,The Rolling Stones,Beggars Banquet (50th Anniversary Edition),1968-12-06,76
68,Fantasy,Mariah Carey,Daydream,1995-10-03,80
4,Tick Tick Boom (feat. BygTwo3) - From 'SCOOB!'...,"Sage The Gemini, BygTwo3",Tick Tick Boom (feat. BygTwo3),2020-05-15,58
88,All The Stars (with SZA),"Kendrick Lamar, SZA",Black Panther The Album Music From And Inspire...,2018-02-09,88
85,Sunflower - Spider-Man: Into the Spider-Verse,"Post Malone, Swae Lee",Hollywood's Bleeding,2019-09-06,88
94,Shivers,Ed Sheeran,Shivers,2021-09-10,1
99,No Lie,"Sean Paul, Dua Lipa",Mad Love The Prequel,2018-06-29,84
9,Girls Just Want to Have Fun,Cyndi Lauper,She's So Unusual,1983-10-14,84
96,Adore You,Harry Styles,Fine Line,2019-12-13,85
